In [9]:
# Import libraries
import psycopg2
import getpass
import numpy as np
import os.path
import datetime

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
port = '5433'
dbname = 'mimic'
schema = 'mimiciii'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, port=port, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))
con.commit()
print("Connected!")

Password: ··········


Connected!


In [4]:
# Load admission_ids
_adm = np.load('res/admission_ids.npy', allow_pickle=True).tolist()
admission_ids = _adm['admission_ids']

# Load item_ids
db = np.load('res/item_ids.npy', allow_pickle=True).tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

print("Loaded!")

Loaded!


In [5]:
# Display all distinct units in OUTPUT table
sql = \
'''
SELECT DISTINCT valueuom 
FROM outputevents
'''
cur.execute(sql)
output_units = cur.fetchall()
print(output_units)

[(None,), ('ml',), ('mL',)]


In [6]:
# All records have the same units, so all item IDs can be saved
valid_output = {'id':output_itemid, 'unit':None}
np.save('res/filtered_output.npy', valid_output)
print('Saved!')

Saved!


In [11]:
# Display all distinct units in OUTPUT table
start = datetime.datetime.now()
output_itemid_units = []
for i in output_itemid:
    sql = \
    f'''
    SELECT valueuom, COUNT(*)
    FROM outputevents
    WHERE itemid={i} AND hadm_id IN (
        SELECT *
        FROM admission_ids)
    GROUP BY valueuom
    '''
    cur.execute(sql)
    res = cur.fetchall()
    output_itemid_units.append(res)
end = datetime.datetime.now()

In [12]:
# Check everything is working
print(end-start)
for i in range(10):
    print(output_itemid_units[i])

0:00:21.180362
[(None, 1), ('ml', 2)]
[('mL', 446)]
[(None, 1), ('ml', 8)]
[(None, 25), ('ml', 219)]
[(None, 3), ('ml', 33)]
[('ml', 1)]
[(None, 1), ('ml', 2)]
[(None, 2), ('ml', 2)]
[('ml', 26)]
[('ml', 10)]
